In [1]:
from genericpath import exists
import os
import sys
import argparse
from datetime import datetime
import numpy as np
import time
from tqdm import tqdm

CONTACT_DIR = '/home/tele_mani_u20/code_proj/cgn/June_12/finetune_cgn_one_object_one_scene/contact_graspnet'
BASE_DIR = os.path.dirname(CONTACT_DIR)
ROOT_DIR = os.path.dirname(os.path.dirname(BASE_DIR))
sys.path.append(os.path.join(BASE_DIR))
sys.path.append(os.path.join(ROOT_DIR))
sys.path.append(os.path.join(BASE_DIR, 'pointnet2',  'models'))
sys.path.append(os.path.join(BASE_DIR, 'pointnet2',  'utils'))

import tensorflow.compat.v1 as tf
tf.disable_eager_execution()
TF2 = True
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)
    
import config_utils
from data import PointCloudReader, load_scene_contacts, center_pc_convert_cam
from summaries import build_summary_ops, build_file_writers
from tf_train_ops import load_labels_and_losses, build_train_op
from contact_grasp_estimator import GraspEstimator

    
parser = argparse.ArgumentParser()
parser.add_argument('--ckpt_dir', default='checkpoints/contact_graspnet', help='Checkpoint dir')
parser.add_argument('--data_path', type=str, default=None, help='Grasp data root dir')
parser.add_argument('--max_epoch', type=int, default=None, help='Epochs to run')
parser.add_argument('--batch_size', type=int, default=None, help='Batch Size during training')
parser.add_argument('--arg_configs', nargs="*", type=str, default=[], help='overwrite config parameters')
FLAGS = parser.parse_args(
    [
        "--ckpt_dir",
        "checkpoints/scene_test_2048_bs3_hor_sigma_001_for_debug",
        "--data_path",
        "reconstructed/",
        "--arg_configs",
        "DATA.num_test_scenes:0",
        "OPTIMIZER.batch_size:2",
        "DATA.scene_contacts_path:scene_contacts",
    ]
)

ckpt_dir = FLAGS.ckpt_dir
if not os.path.exists(ckpt_dir): 
    if not os.path.exists(os.path.dirname(ckpt_dir)):
        ckpt_dir = os.path.join(BASE_DIR, ckpt_dir)
    os.makedirs(ckpt_dir, exist_ok=True)

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"

global_config = config_utils.load_config(ckpt_dir, batch_size=FLAGS.batch_size, max_epoch=FLAGS.max_epoch, 
                                        data_path= FLAGS.data_path, arg_configs=FLAGS.arg_configs, save=True)


2022-06-12 14:48:40.325882: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2022-06-12 14:48:40.347466: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:65:00.0 name: NVIDIA TITAN Xp computeCapability: 6.1
coreClock: 1.582GHz coreCount: 30 deviceMemorySize: 11.91GiB deviceMemoryBandwidth: 510.07GiB/s
2022-06-12 14:48:40.347645: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2022-06-12 14:48:40.349155: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2022-06-12 14:48:40.350706: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10
2022-06-12 14:48:40.351110: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcurand.so.10


In [2]:
global_config['DATA']['data_path'] = '../reconstructed'

contact_infos = load_scene_contacts(global_config['DATA']['data_path'],
                                    scene_contacts_path=global_config['DATA']['scene_contacts_path'])

# print(contact_infos)

# global_config['DATA']['scene_contacts_path']

In [3]:

num_train_samples = len(contact_infos)-global_config['DATA']['num_test_scenes']
num_test_samples = global_config['DATA']['num_test_scenes']
    
print('using %s meshes' % (num_train_samples + num_test_samples))
if 'train_and_test' in global_config['DATA'] and global_config['DATA']['train_and_test']:
    num_train_samples = num_train_samples + num_test_samples
    num_test_samples = 0
    print('using train and test data')

pcreader = PointCloudReader(
    root_folder=global_config['DATA']['data_path'],
    batch_size=global_config['OPTIMIZER']['batch_size'],
    estimate_normals=global_config['DATA']['input_normals'],
    raw_num_points=global_config['DATA']['raw_num_points'],
    use_uniform_quaternions = global_config['DATA']['use_uniform_quaternions'],
    scene_obj_scales = [c['obj_scales'] for c in contact_infos],
    scene_obj_paths = [c['obj_paths'] for c in contact_infos],
    scene_obj_transforms = [c['obj_transforms'] for c in contact_infos],
    num_train_samples = num_train_samples,
    num_test_samples = num_test_samples,
    use_farthest_point = global_config['DATA']['use_farthest_point'],
    intrinsics=global_config['DATA']['intrinsics'],
    # The default value in the init fn is equal to default value in yaml file
    # elevation=global_config['DATA']['view_sphere']['elevation'],
    # distance_range=global_config['DATA']['view_sphere']['distance_range'],
    pc_augm_config=global_config['DATA']['pc_augm'],
    depth_augm_config=global_config['DATA']['depth_augm'],
)

with tf.Graph().as_default():
    
    # Build the model
    grasp_estimator = GraspEstimator(global_config)
    ops = grasp_estimator.build_network()
    
    # contact_tensors = load_contact_grasps(contact_infos, global_config['DATA'])
    
    loss_ops = load_labels_and_losses(grasp_estimator, contact_infos, global_config)

    ops.update(loss_ops)
    ops['train_op'] = build_train_op(ops['loss'], ops['step'], global_config)

    # Add ops to save and restore all the variables.
    saver = tf.train.Saver(save_relative_paths=True, keep_checkpoint_every_n_hours=4)

    # Create a session
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    config.allow_soft_placement = True
    sess = tf.Session(config=config)

    # Log summaries
    summary_ops = build_summary_ops(ops, sess, global_config)

    trainable_vars = tf.get_collection_ref(tf.GraphKeys.TRAINABLE_VARIABLES)
    all_vars = tf.get_collection_ref(tf.GraphKeys.GLOBAL_VARIABLES)
    saveable_vars = tf.get_collection_ref(tf.GraphKeys.SAVEABLE_OBJECTS)

using 1 meshes
/home/tele_mani_u20/code_proj/cgn/June_12/finetune_cgn_one_object_one_scene/pointnet2/tf_ops/sampling
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
--- Get model
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.BatchNormalization` documentation).
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
total positive contact points  4
grasp label device:  /cpu:0
Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_initializabl

2022-06-12 14:48:49.685472: I tensorflow/core/platform/cpu_feature_guard.cc:143] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2022-06-12 14:48:49.710565: I tensorflow/core/platform/profile_utils/cpu_utils.cc:102] CPU Frequency: 3299990000 Hz
2022-06-12 14:48:49.711240: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f34c4000b60 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-06-12 14:48:49.711260: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2022-06-12 14:48:49.767626: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5641c1e8d230 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2022-06-12 14:48:49.767653: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA TITAN Xp, Compute Capability 6.1
2022-06-12 14:48:49.768157: I tensorflow

In [4]:
saved_variables = tf.train.list_variables(ckpt_dir)
saved_variables[200]

('layer1/conv0_0/bn/beta', [32])

In [5]:
len(saved_variables), len(trainable_vars), len(saveable_vars)

(617, 173, 0)

In [6]:
# https://stackoverflow.com/questions/40925652/in-tensorflow-whats-the-meaning-of-0-in-a-variables-name

# So for all the trainable variables, 
# they are saved and can be restored

# The additional variables that can not be restored have the form
# moving_*/Adan*
# vars = trainable_vars
# vars = all_vars
vars = saveable_vars

saved_var_names = list(v[0] for v in saved_variables)

In [7]:
for v in vars:

    n = v.name.strip(':0')

    if n not in saved_var_names:

        
        print(n)

In [8]:
for v in saved_variables:

    if v not in trainable_vars:
        print(v)

('Variable', [])
('beta1_power', [])
('beta2_power', [])
('fa_layer1/conv_0/biases', [256])
('fa_layer1/conv_0/biases/Adam', [256])
('fa_layer1/conv_0/biases/Adam_1', [256])
('fa_layer1/conv_0/bn/beta', [256])
('fa_layer1/conv_0/bn/beta/Adam', [256])
('fa_layer1/conv_0/bn/beta/Adam_1', [256])
('fa_layer1/conv_0/bn/gamma', [256])
('fa_layer1/conv_0/bn/gamma/Adam', [256])
('fa_layer1/conv_0/bn/gamma/Adam_1', [256])
('fa_layer1/conv_0/bn/moving_mean', [256])
('fa_layer1/conv_0/bn/moving_variance', [256])
('fa_layer1/conv_0/weights', [1, 1, 1664, 256])
('fa_layer1/conv_0/weights/Adam', [1, 1, 1664, 256])
('fa_layer1/conv_0/weights/Adam_1', [1, 1, 1664, 256])
('fa_layer1/conv_1/biases', [256])
('fa_layer1/conv_1/biases/Adam', [256])
('fa_layer1/conv_1/biases/Adam_1', [256])
('fa_layer1/conv_1/bn/beta', [256])
('fa_layer1/conv_1/bn/beta/Adam', [256])
('fa_layer1/conv_1/bn/beta/Adam_1', [256])
('fa_layer1/conv_1/bn/gamma', [256])
('fa_layer1/conv_1/bn/gamma/Adam', [256])
('fa_layer1/conv_1/bn

In [10]:
restorable_variables = []

for v in all_vars:

    if v.name.strip(':0') in saved_var_names:
        restorable_variables.append(v)

    else:
        print(v.name)

In [12]:
len(restorable_variables), len(saved_var_names), len(all_vars)

(601, 617, 601)

In [16]:
restorable_var_names = list(v.name for v in restorable_variables)

for v in saved_var_names:

    v += ':0'

    if v not in restorable_var_names:
        print(v)

fc1/bn/moving_mean/Adam:0
fc1/bn/moving_mean/Adam_1:0
fc1/bn/moving_variance/Adam:0
fc1/bn/moving_variance/Adam_1:0
fc1_app/bn/moving_mean/Adam:0
fc1_app/bn/moving_mean/Adam_1:0
fc1_app/bn/moving_variance/Adam:0
fc1_app/bn/moving_variance/Adam_1:0
fc1_off/bn/moving_mean/Adam:0
fc1_off/bn/moving_mean/Adam_1:0
fc1_off/bn/moving_variance/Adam:0
fc1_off/bn/moving_variance/Adam_1:0
fc1_seg/bn/moving_mean/Adam:0
fc1_seg/bn/moving_mean/Adam_1:0
fc1_seg/bn/moving_variance/Adam:0
fc1_seg/bn/moving_variance/Adam_1:0
